### Imports

In [4]:
import matplotlib.pyplot as plt
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from IPython.display import display
from matplotlib.cm import plasma
from matplotlib.colors import Normalize

from src.rft_main import *


### Inputs

In [5]:
### INPUTS ###
inputs = {
    "model": "CylinderRough",
    "bulk_density": 1310,
    "friction_material": 0.21,
    "firction_surface": 0.4,
    "friction_type": "coefficient",
    "gravity": 9.81,
    "linear_velocity": 0.1,
    "direction_angle_xz_deg": -90,
    "direction_angle_y_deg": -90,
    "rotation": True,
    "angular_velocity": np.array([0, 0, -2 * np.pi]),
    "start_depth": 10,
    "end_depth": 125,
    "step_size": 5,
}
colormap = "Viridis"


In [6]:
results = run_rft(**inputs)

point_list = results["point_list"]
normal_list = results["normal_list"]
area_list = results["area_list"]
depth_list = results["depth_list"]
object_width_x = results["object_width_x"]
object_width_y = results["object_width_y"]
object_height = results["object_height"]
vertices = results["vertices"]
faces = results["faces"]
trg = results["trg"]
movement = results["movement"]
z_local = results["z_local"]
r_local = results["r_local"]
theta_local = results["theta_local"]
alpha_generic = results["alpha_generic"]
alpha_generic_n = results["alpha_generic_n"]
alpha_generic_t = results["alpha_generic_t"]
alpha = results["alpha"]
depth = results["depth"]
forces = results["forces"]
pressures = results["pressures"]
force_x = results["force_x"]
force_y = results["force_y"]
force_z = results["force_z"]
resultant = results["resultant"]
torques = results["torques"]
torque_x = results["torque_x"]
torque_y = results["torque_y"]
torque_z = results["torque_z"]
resultant_torque = results["resultant_torque"]
result_matrix = results["result_matrix"]
print(force_x, force_y, force_z)
print("Done!")


Processed movement at depth: 10 mm


ValueError: operands could not be broadcast together with shapes (902,3) (4272,3) 

In [ ]:
%%html
<style>
.cell-output-ipywidget-background {
   background-color: transparent !important;
}
.jp-OutputArea-output {
   background-color: transparent;
}
</style>


# Plots

## 3D Mesh

In [ ]:
# Creating a 3D plot using Plotly
fig = go.FigureWidget()

fig.add_trace(
    go.Mesh3d(
        x=vertices[:, 0],
        y=vertices[:, 1],
        z=vertices[:, 2],
        i=faces[:, 0],
        j=faces[:, 1],
        k=faces[:, 2],
        colorscale=colormap,
        showscale=False,
        intensity=vertices[:, 2],
        cmin=np.min(vertices[:, 2]),
        cmax=np.max(vertices[:, 2]),
    )
)

# Updating the layout of the plot
fig.update_layout(
    width=600,
    height=500,
    margin=dict(l=10, r=10, t=10, b=10),
    scene=dict(
        aspectmode="data",
        aspectratio=dict(
            x=np.ptp(point_list[:, 0]),
            y=np.ptp(point_list[:, 1]),
            z=np.ptp(point_list[:, 2]),
        ),
        camera=dict(eye=dict(x=1.75, y=1.5, z=1.25)),
    ),
)

display(fig)


## Movement

In [ ]:
fig = go.FigureWidget()
fig.add_trace(
    go.Cone(
        x=point_list[:, 0],
        y=point_list[:, 1],
        z=point_list[:, 2],
        u=movement[:, 0],
        v=movement[:, 1],
        w=movement[:, 2],
        sizeref=3,
        showscale=False,
    )
)

# Updating the layout of the plot
fig.update_layout(
    width=600,
    height=500,
    margin=dict(l=10, r=10, t=10, b=10),
    scene=dict(
        aspectmode="data",
        aspectratio=dict(
            x=np.ptp(point_list[:, 0]),
            y=np.ptp(point_list[:, 1]),
            z=np.ptp(point_list[:, 2]),
        ),
        camera=dict(eye=dict(x=1.75, y=1.5, z=1.25)),
    ),
)

display(fig)


## Force Quiver

In [ ]:
# Calculating the endpoints and the magnitudes of the vectors
endpoints = point_list - forces * 200
magnitudes = np.linalg.norm(forces, axis=1)
# Normalize the magnitudes to the range [0, 1]
norm = Normalize()
normalized_magnitudes = norm(magnitudes)
# Get colors from a colormap
colors = plasma(normalized_magnitudes)

# Preparing data for plot
x_lines = np.c_[
    point_list[:, 0], endpoints[:, 0], np.full(len(point_list), np.nan)
].flatten()
y_lines = np.c_[
    point_list[:, 1], endpoints[:, 1], np.full(len(point_list), np.nan)
].flatten()
z_lines = np.c_[
    point_list[:, 2], endpoints[:, 2], np.full(len(point_list), np.nan)
].flatten()

# Flatten the colors array and convert to RGB strings
colors = (colors[:, :3] * 255).astype(int)
color_strings = [f"rgb({r},{g},{b})" for r, g, b in colors]
color_strings = np.repeat(
    color_strings, 3
)  # Repeat for each segment of the line (start, end, break)

# Creating a 3D plot using Plotly
line_marker = dict(width=5, color=color_strings)
fig = go.FigureWidget(
    data=[go.Scatter3d(x=x_lines, y=y_lines, z=z_lines, mode="lines", line=line_marker)]
)

# Updating the layout of the plot
fig.update_layout(
    width=600,
    height=500,
    margin=dict(l=10, r=10, t=10, b=10),
    scene=dict(
        aspectmode="data",
        aspectratio=dict(x=1, y=1, z=1),
        camera=dict(eye=dict(x=1.75, y=1.5, z=1.25)),
    ),
)

# Show the plot
display(fig)


## Force Scatter

In [ ]:
magnitudes = np.linalg.norm(pressures, axis=1)

fig = go.Figure()

scatter = go.Scatter3d(
    x=point_list[:, 0],
    y=point_list[:, 1],
    z=point_list[:, 2],
    mode="markers",
    marker=dict(
        size=2,
        color=magnitudes,  # Set color equal to a variable
        colorscale=colormap,  # Choose a colormap
        colorbar=dict(
            title="Pressure [N/mm²]",
            len=0.75,
            thickness=20,
        ),
        opacity=1,
    ),
)

fig.add_trace(scatter)

# Update the layout if needed
fig.update_layout(
    width=700,
    height=500,
    margin=dict(l=10, r=10, t=10, b=10),
    scene=dict(
        aspectmode="data",
        aspectratio=dict(x=1, y=1, z=1),
        camera=dict(eye=dict(x=2, y=2, z=1.5)),
    ),
)

# Show the plot
display(fig)


## Force Scatter XYZ

In [ ]:
# Calculate global color scale limits based on the maximum absolute pressure value
max_abs_pressure = np.max(np.abs(pressures))

fig = make_subplots(
    rows=1,
    cols=3,
    subplot_titles=("X", "Y", "Z"),
    specs=[[{"type": "scatter3d"}, {"type": "scatter3d"}, {"type": "scatter3d"}]],
)

for i in range(3):
    scatter = go.Scatter3d(
        x=point_list[:, 0],
        y=point_list[:, 1],
        z=point_list[:, 2],
        mode="markers",
        marker=dict(
            size=2,
            color=np.abs(pressures[:, i]),  # Set color equal to a variable
            colorscale=colormap,  # Choose a colormap
            cmin=0,  # Set consistent color scale limits
            cmax=max_abs_pressure,  # Set consistent color scale limits
            colorbar=dict(
                title="Pressure [N/mm²]",
                len=0.75,
                thickness=20,
            ),
            opacity=1,
        ),
    )
    fig.add_trace(scatter, row=1, col=i + 1)

fig.update_layout(
    width=1200,  # Adjusted for 3 plots
    height=500,
    margin=dict(l=10, r=10, t=30, b=10),
    scene1=dict(
        aspectmode="data",
        aspectratio=dict(x=1, y=1, z=1),
        camera=dict(eye=dict(x=3, y=3, z=2)),
    ),
    scene2=dict(
        aspectmode="data",
        aspectratio=dict(x=1, y=1, z=1),
        camera=dict(eye=dict(x=3, y=3, z=2)),
    ),
    scene3=dict(
        aspectmode="data",
        aspectratio=dict(x=1, y=1, z=1),
        camera=dict(eye=dict(x=3, y=3, z=2)),
    ),
)

display(fig)


In [ ]:
fig = make_subplots(
    rows=2,
    cols=3,
    subplot_titles=("X", "Y", "Z"),
)

for i in range(3):
    scatter = go.Scatter(
        x=result_matrix[:, 0],
        y=result_matrix[:, i + 1],
        name="spline",
        text=["tweak line smoothness<br>with 'smoothing' in line object"],
        hoverinfo="text+name",
        line_shape="spline",
    )
    fig.add_trace(scatter, row=1, col=i + 1)

fig.update_layout(
    width=1200,  # Adjusted for 6 plots
    height=600,
    margin=dict(l=10, r=10, t=30, b=10),
)

display(fig)
